# Generate a PowerPoint With an LLM

This is the notebook related to the Medium article *[How to Generate a PowerPoint With an LLM](https://medium.com/@matteo28/how-to-generate-a-powerpoint-with-an-llm-3b0448a48125)*.

In this notebook there is code to be able to create a powerpoint using the Gemini to take advantage of the Gemini-2.0 templates for text generation and imaged-3 for image generation.

In [1]:
!pip install llama-index-llms-google-genai llama-index


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:

import re

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

SVC_ACC = !(gcloud config get-value core/account)
SVC_ACC = SVC_ACC[0]

PROJECT_NUMBER=str(re.search(r'\d+', SVC_ACC).group())

LOCATION="us-central1"

FOLDER_NAME="."

In [4]:
import requests
import json
import os
import re
from llama_index.core import PromptTemplate


from llama_index.llms.google_genai import GoogleGenAI

llm = GoogleGenAI(
    model="gemini-2.0-flash",
    vertexai_config={"project": PROJECT_ID, "location": LOCATION},
)


from pptx import Presentation
from config import Config

## Load template

In [5]:
template_path = os.path.join(Config.TEMPLATES_PATH, 'basic_template.pptx')
prs = Presentation(template_path)

In [6]:
slide_structures = []

# Iterate through each slide in the presentation
for slide in prs.slides:
    structure = {"placeholders": []}
    
    # Iterate through each shape in the slide
    for shape in slide.shapes:
        if shape.is_placeholder:
            
            # Create a dictionary to store placeholder details
            placeholder = {
                "type": shape.placeholder_format.type,
                "idx": shape.placeholder_format.idx,
                "has_text_frame": shape.has_text_frame,
                "name": shape.name,
            }
            # Check if the placeholder contains a table
            if shape.has_table:
                placeholder["has_table"] = True
                # Extract the table structure as a list of rows with cell texts
                placeholder["table_structure"] = [
                    [cell.text for cell in row.cells] for row in shape.table.rows
                ]
            else:
                placeholder["has_table"] = False
            
            # Check if the placeholder is an image placeholder
            if shape.placeholder_format.type == 18:
                placeholder["has_image"] = True
                placeholder["image_description"] = ""
            else:
                placeholder["has_image"] = False

            # Append the placeholder information to the structure
            structure["placeholders"].append(placeholder)
            
    # Append the slide structure to the list of all slide structures
    slide_structures.append(structure)

# Convert the structure list to a JSON string for better readability
slide_structures_str = json.dumps(slide_structures, indent=4)

In [7]:
import pprint
pprint.pprint(slide_structures_str)

('[\n'
 '    {\n'
 '        "placeholders": [\n'
 '            {\n'
 '                "type": 1,\n'
 '                "idx": 0,\n'
 '                "has_text_frame": true,\n'
 '                "name": "Title 1",\n'
 '                "has_table": false,\n'
 '                "has_image": false\n'
 '            }\n'
 '        ]\n'
 '    },\n'
 '    {\n'
 '        "placeholders": [\n'
 '            {\n'
 '                "type": 1,\n'
 '                "idx": 0,\n'
 '                "has_text_frame": true,\n'
 '                "name": "Title 1",\n'
 '                "has_table": false,\n'
 '                "has_image": false\n'
 '            },\n'
 '            {\n'
 '                "type": 7,\n'
 '                "idx": 15,\n'
 '                "has_text_frame": true,\n'
 '                "name": "Content Placeholder 2",\n'
 '                "has_table": false,\n'
 '                "has_image": false\n'
 '            },\n'
 '            {\n'
 '                "type": 18,\n'
 '          

## Prompting

In [8]:
prompt = """
You are a PowerPoint presentation specialist. You are asked to create content for a presentation about {topic}.
The PowerPoint template has the following structure:

{structure}

You need to generate content that fits into this structure, ensuring that all placeholders are filled appropriately.

For each slide:
1. Provide the title for the slide.
2. Provide the text content for each text placeholder.
3. If a slide contains a table, generate appropriate data to fill it based on the provided context.
4. If a slide contains an image placeholder, describe the type of image that should be included.

Return the structured information *only* as a JSON. Do not include any introductory text or explanations.
"""

prompt_template = PromptTemplate(template=prompt)

test_topic = "Create an example of pitch for a new product named \"Nose\", a platform that helps investors to find the best investment opportunities."

content_prompt = (
    prompt_template.format(topic=test_topic, structure=slide_structures_str)
)

## LLM

In [9]:
from crewai import Agent, Task, Crew, LLM

# llm = LLM(
#               model='vertex_ai/gemini-2.0-flash',
#               vertex_project = PROJECT_ID
#             )

slides_response = llm.complete(content_prompt, True).text

In [10]:
print(slides_response)

```json
[
  {
    "placeholders": [
      {
        "type": 1,
        "idx": 0,
        "has_text_frame": true,
        "name": "Title 1",
        "has_table": false,
        "has_image": false,
        "text": "Nose: Your Sixth Sense for Investment Opportunities"
      }
    ]
  },
  {
    "placeholders": [
      {
        "type": 1,
        "idx": 0,
        "has_text_frame": true,
        "name": "Title 1",
        "has_table": false,
        "has_image": false,
        "text": "The Problem: Finding the Right Investments is Hard"
      },
      {
        "type": 7,
        "idx": 15,
        "has_text_frame": true,
        "name": "Content Placeholder 2",
        "has_table": false,
        "has_image": false,
        "text": "- Information overload: Too much data, not enough insight.\n- Missed opportunities: Hidden gems overlooked due to lack of resources.\n- Time-consuming research: Sifting through countless reports and articles.\n- Inefficient networking: Difficulty connecting w

In [11]:
json_match = re.search(r'```json\s*(.*?)\s*```', slides_response, re.DOTALL)

if json_match:
    json_content = json_match.group(1)
    try:
        slides = json.loads(json_content)
    except json.JSONDecodeError as e:
        raise ValueError(f"JSON parsing error: {str(e)}")
else:
    raise ValueError("The JSON was not found in the model response.")

In [12]:
pprint.pprint(slides)

[{'placeholders': [{'has_image': False,
                    'has_table': False,
                    'has_text_frame': True,
                    'idx': 0,
                    'name': 'Title 1',
                    'text': 'Nose: Your Sixth Sense for Investment '
                            'Opportunities',
                    'type': 1}]},
 {'placeholders': [{'has_image': False,
                    'has_table': False,
                    'has_text_frame': True,
                    'idx': 0,
                    'name': 'Title 1',
                    'text': 'The Problem: Finding the Right Investments is '
                            'Hard',
                    'type': 1},
                   {'has_image': False,
                    'has_table': False,
                    'has_text_frame': True,
                    'idx': 15,
                    'name': 'Content Placeholder 2',
                    'text': '- Information overload: Too much data, not enough '
                            'ins

In [13]:
from google import genai

def generate_image(description):
    response = genai.images.generate(
        model="imagen-e-3",
        prompt=description,
        n=1,
        response_format="url",
    )
    image_url = response.data[0].url
    image_data = requests.get(image_url).content
    return image_data

In [14]:
def populate_slide(slide, slide_data):
    # Get the list of placeholders
    placeholders = slide_data.get("placeholders", [])

    # Popolare i placeholders di testo
    for placeholder in placeholders:
        for shape in slide.shapes:
            if shape.is_placeholder and placeholder.get("name") in shape.name:
                if placeholder.get("text") and shape.has_text_frame:
                    shape.text = placeholder["text"]

    # Populate text placeholders
    for placeholder in placeholders:
        if placeholder.get("image_description"):
            # image_data = generate_image(placeholder["image_description"])
            img_path = "temp_image.png"
            # with open(img_path, "wb") as img_file:
            #     img_file.write(image_data)
            # for shape in slide.shapes:
            #     if shape.is_placeholder and shape.placeholder_format.type == 18:
            #         slide.shapes._spTree.remove(shape._element)
            #         slide.shapes.add_picture(img_path, shape.left, shape.top, shape.width, shape.height)
            #         break

    # Populate the tables
    for placeholder in placeholders:
        table_data = placeholder.get("table_structure")
        if table_data:
            for shape in slide.shapes:
                if shape.has_table:
                    table = shape.table
                    for i, row_data in enumerate(table_data):
                        for j, cell_text in enumerate(row_data):
                            table.cell(i, j).text = cell_text
                    break

In [15]:
for i, slide_data in enumerate(slides):
    if i < len(prs.slides):
        populate_slide(prs.slides[i], slide_data)

In [16]:
save_path = os.path.join(Config.POWERPOINTS_PATH, "output2.pptx")
prs.save(save_path)

In [17]:
save_path

'/home/jupyter/GenAI9/llm-powerpoint/powerpoints/output2.pptx'